# Classification of Proton Decay Events vs atm $\nu$ Background

Proton decay events are simulates in the Super-Kamiokande Experiment. In particular, the prompt gamma mode where the proton decays in the SUSY favored mode: $p\rightarrow \bar{\nu}K^{+}$, the Kaon decays leptonically: $K^{+}\rightarrow \nu \mu^{+}$, and the remaining nucleus emits a low energy deexcitation gamma.
Background events for this search come from neutrinos produced in Earth's atmosphere that interact inside the detector faking a proton decay event.

Import the necessary modules for the analysis

In [51]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Read the data, with format given by columns:

In [2]:
files = '/Users/santucci/Dropbox/DataScience/Udacity/ML_nanodegree/P5_Capstone/capstone/project/data/pdkdata_{}.csv'
pdk_file = files.format(1)
atm_file = files.format(0)

In [18]:
columns = """pmu,
pgamma,
deltat,
pmgnll,
enll,
munll,
pinll,
open_ang,
open_angElGamma,
open_angMuEl,
impact,
dist,
dist0,
pi0nll,
ppi01,
ppi02,
ppi0,
pi0t0,
mpi0,
pi0ang,
pms,
pmichel"""

In [19]:
names = columns.split('\n')
names = [i[:-1] if i!='pmichel' else i for i in names]

In [44]:
pdk_total = pd.read_csv(pdk_file, names=names)
atm_total = pd.read_csv(atm_file, names=names)

Basic inspection of data to see if it was read properly: 

In [45]:
pdk_total.head()

,pmu,pgamma,deltat,pmgnll,enll,munll,pinll,open_ang,open_angElGamma,open_angMuEl,...,dist0,pi0nll,ppi01,ppi02,ppi0,pi0t0,mpi0,pi0ang,pms,pmichel
0,241.391,6.26128,14.613000,2558.25,2914.04,2664.15,2645.14,2.277740,1.99308,0.750705,...,29.6624,2855.03,20.7505,18.994700,38.0401,963.089,11.516500,0.588542,242.413,36.2222
1,231.414,2.20592,1.965090,3305.63,3567.35,3339.40,3336.74,2.721860,1.06760,2.477210,...,29.1421,3549.69,41.9162,1.642250,43.4761,930.614,2.678610,0.324267,230.885,18.4711
2,233.476,3.22373,0.177734,2002.50,2113.08,2022.08,2008.93,0.175061,2.19556,2.077550,...,61.0677,2093.94,41.5561,0.555372,42.1030,973.058,0.842795,0.175659,234.335,31.0928
3,245.813,5.20290,1.908630,3974.67,4257.48,4013.86,3992.31,0.860520,1.20056,0.413600,...,72.9799,4148.75,54.4148,7.265250,61.2030,963.632,7.656890,0.387516,246.149,33.5857
4,240.372,5.97949,4.830750,2809.95,3046.41,2889.52,2863.53,2.056880,1.48716,1.861790,...,63.2966,3017.51,39.3333,6.235480,45.1151,966.336,6.414010,0.412475,239.426,40.0410


In [46]:
atm_total.head()

,pmu,pgamma,deltat,pmgnll,enll,munll,pinll,open_ang,open_angElGamma,open_angMuEl,...,dist0,pi0nll,ppi01,ppi02,ppi0,pi0t0,mpi0,pi0ang,pms,pmichel
0,228.749,4.20081,0.371948,2730.35,2796.10,2742.23,2732.36,1.18532,2.181390,1.139380,...,106.7750,2779.53,39.5197,1.808540,41.2905,976.278,1.765960,0.209267,230.501,43.55540
1,246.664,6.34188,1.610780,2621.84,2863.00,2698.61,2681.86,1.59260,0.629466,0.972799,...,31.2172,2842.83,48.3781,0.596231,48.9653,972.821,0.937224,0.174729,246.678,28.98000
2,242.924,2.75621,1.128050,2266.33,2363.87,2271.03,2255.27,1.85268,2.661000,1.003350,...,50.8724,2346.48,57.2483,0.597311,57.7841,980.979,2.667540,0.460223,247.482,25.12600
3,258.500,6.82106,3.373110,4294.38,4678.36,4398.24,4373.70,1.33615,1.015790,0.567112,...,140.9680,4644.29,36.6052,32.691500,68.6906,936.105,9.145360,0.265146,258.232,22.17040
4,245.857,6.48899,0.232117,3470.27,3698.13,3518.47,3493.29,2.07804,1.312920,1.227750,...,82.9542,3680.28,56.6713,0.578721,57.2463,939.141,0.657577,0.114887,247.221,4.89588


Data cleaning to only use certain features and some new features are created. Namely, the class type depending if it is a proton decay simulation or atmospheric neutrinos. Also, likelihood ratios are defined.

In [47]:
pdk = pdk_total.drop(['pmgnll', 'enll', 'munll', 'pinll', 'ppi01', 'ppi02', 'pi0t0','mpi0', 'pi0ang', 'pms', 'pmichel', 'open_angElGamma', 'open_angMuEl'],1)
pdk['mu_e'] = pdk_total['enll'] - pdk_total['munll']
pdk['pmg_mu'] = pdk_total['munll'] - pdk_total['pmgnll']
pdk['pi_mu'] = pdk_total['munll'] - pdk_total['pinll']
pdk['Class'] = 1

In [48]:
atm = atm_total.drop(['pmgnll', 'enll', 'munll', 'pinll', 'ppi01', 'ppi02', 'pi0t0','mpi0', 'pi0ang', 'pms', 'pmichel', 'open_angElGamma', 'open_angMuEl'],1)
atm['mu_e'] = atm_total['enll'] - atm_total['munll']
atm['pmg_mu'] = atm_total['munll'] - atm_total['pmgnll']
atm['pi_mu'] = atm_total['munll'] - atm_total['pinll']
atm['Class'] = 0

In [49]:
pdk.head()

,pmu,pgamma,deltat,open_ang,impact,dist,dist0,pi0nll,ppi0,mu_e,pmg_mu,pi_mu,Class
0,241.391,6.26128,14.613000,2.277740,14.8869,35.8988,29.6624,2855.03,38.0401,249.89,105.90,19.01,1
1,231.414,2.20592,1.965090,2.721860,16.7900,34.4681,29.1421,3549.69,43.4761,227.95,33.77,2.66,1
2,233.476,3.22373,0.177734,0.175061,21.7911,21.9034,61.0677,2093.94,42.1030,91.00,19.58,13.15,1
3,245.813,5.20290,1.908630,0.860520,51.8033,52.5568,72.9799,4148.75,61.2030,243.62,39.19,21.55,1
4,240.372,5.97949,4.830750,2.056880,52.6895,57.4101,63.2966,3017.51,45.1151,156.89,79.57,25.99,1


In [50]:
atm.head()

,pmu,pgamma,deltat,open_ang,impact,dist,dist0,pi0nll,ppi0,mu_e,pmg_mu,pi_mu,Class
0,228.749,4.20081,0.371948,1.18532,68.24280,74.2964,106.7750,2779.53,41.2905,53.87,11.88,9.87,0
1,246.664,6.34188,1.610780,1.59260,28.50080,55.7484,31.2172,2842.83,48.9653,164.39,76.77,16.75,0
2,242.924,2.75621,1.128050,1.85268,7.49464,11.4690,50.8724,2346.48,57.7841,92.84,4.70,15.76,0
3,258.500,6.82106,3.373110,1.33615,37.52420,79.3712,140.9680,4644.29,68.6906,280.12,103.86,24.54,0
4,245.857,6.48899,0.232117,2.07804,72.87320,75.7447,82.9542,3680.28,57.2463,179.66,48.20,25.18,0


Merge both data sets into one data frame and perform basic sanity check:

In [54]:
data = pd.concat([pdk, atm], ignore_index=True)

In [55]:
data.head()

,pmu,pgamma,deltat,open_ang,impact,dist,dist0,pi0nll,ppi0,mu_e,pmg_mu,pi_mu,Class
0,241.391,6.26128,14.613000,2.277740,14.8869,35.8988,29.6624,2855.03,38.0401,249.89,105.90,19.01,1
1,231.414,2.20592,1.965090,2.721860,16.7900,34.4681,29.1421,3549.69,43.4761,227.95,33.77,2.66,1
2,233.476,3.22373,0.177734,0.175061,21.7911,21.9034,61.0677,2093.94,42.1030,91.00,19.58,13.15,1
3,245.813,5.20290,1.908630,0.860520,51.8033,52.5568,72.9799,4148.75,61.2030,243.62,39.19,21.55,1
4,240.372,5.97949,4.830750,2.056880,52.6895,57.4101,63.2966,3017.51,45.1151,156.89,79.57,25.99,1


In [56]:
data.tail()

,pmu,pgamma,deltat,open_ang,impact,dist,dist0,pi0nll,ppi0,mu_e,pmg_mu,pi_mu,Class
52970,247.648,6.65141,1.777590,1.425790,59.1203,59.1217,84.7140,5009.73,61.1620,242.07,91.95,29.05,0
52971,220.567,4.54587,1.746580,0.659527,27.9286,27.9716,57.8948,3025.70,40.6072,165.24,32.85,23.90,0
52972,260.676,5.57783,2.389830,2.691140,16.8382,25.5875,50.5605,4586.94,69.8079,198.58,67.60,15.27,0
52973,212.778,5.97918,0.115295,0.932033,27.2751,55.1514,97.7204,2573.98,33.7420,100.53,48.72,14.29,0
52974,218.726,3.55055,3.441100,1.065330,36.6207,63.6685,36.8061,3208.61,40.0615,111.76,32.39,15.53,0


In [68]:
len(data) == len(pdk) + len(atm)

True

In [77]:
features = data.drop(['Class'],1)
labels = data['Class']

In [78]:
from sklearn.tree import DecisionTreeClassifier as Tree

In [79]:
tree = Tree(criterion='gini', splitter='best', max_depth=None, min_samples_split=.1, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=42)

In [80]:
def split_data(X, y):
    from sklearn.cross_validation import train_test_split
    return train_test_split(X, y, test_size=0.2, random_state = 42)

X_train, X_test, y_train, y_test = split_data(features, labels)

/Users/santucci/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [81]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=0.1, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best')

In [83]:
tree.score(X_test, y_test)

0.8617272298253893

In [85]:
y_pred = tree.predict(X_test)

In [89]:
sum(y_pred)

5256

In [88]:
sum(y_test - y_pred == -1)

527

In [82]:
from IPython.display import Image  
dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=iris.feature_names,  
                         class_names=iris.target_names,  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png())  

AttributeError: 'DecisionTreeClassifier' object has no attribute 'export_graphviz'